# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201202_094646_cgan_model2', '20201207_091602_lambd…

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201208_161644_lambda0.1


In [11]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
62491,62491.0,19.0,0.075227,0.108568,0.183795,4.096621,86.374359,82.277740,-1.875954,88.194931,-1.976739,4.457255,-4.187650,-4.065115,0.137172
62492,62492.0,19.0,0.105527,0.148155,0.253682,6.731944,86.316040,79.584099,-1.888642,88.827759,-1.851057,4.830481,-3.048982,-6.728847,0.133656
62493,62493.0,19.0,0.152817,0.067759,0.220576,5.313069,80.811188,75.498116,-2.117865,84.180145,-1.888726,3.191310,-5.988683,-5.295366,0.133734
62494,62494.0,19.0,0.070539,0.142038,0.212576,4.725043,81.156013,76.430969,-2.430514,86.989349,-2.274181,4.570160,-4.484430,-4.701867,0.134452
62495,62495.0,19.0,0.110034,0.129804,0.239837,5.686406,85.618492,79.932083,-2.318186,80.678497,-2.622641,5.093661,-3.330529,-5.679826,0.134384
62496,62496.0,19.0,0.188812,0.076574,0.265386,4.518950,81.868507,77.349556,-2.313645,86.664673,-2.394696,3.415005,-4.611236,-4.490138,0.137818
62497,62497.0,19.0,0.100619,0.087172,0.187791,4.666985,84.378761,79.711777,-2.508943,78.897141,-2.610133,4.535168,-3.564695,-4.642609,0.133608
62498,62498.0,19.0,0.101601,0.195551,0.297152,5.606174,85.849976,80.243805,-2.588088,85.620453,-2.499305,4.140037,-3.749785,-5.596510,0.133119
62499,62499.0,19.0,0.125286,0.128336,0.253622,5.201418,83.829910,78.628494,-2.460347,80.520447,-2.610785,4.425801,-4.502185,-5.189441,0.134506
62500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.902130,-2.338282,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-2.9784140133857724


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
58343,58343.0,18.0,0.119685,0.109013,0.228699,5.949080,85.189095,79.240013,-2.902843,81.792953,-3.210074,6.080492,-4.639206,-5.941076,0.135599
58877,58877.0,18.0,0.136434,0.233085,0.369519,5.046766,81.702965,76.656197,-3.139060,85.280731,-3.208382,5.029647,-2.584076,-5.034982,0.131992
61102,61102.0,19.0,0.115517,0.083937,0.199455,5.460420,84.645813,79.185394,-3.132429,84.146347,-3.206190,5.837498,-5.359791,-5.452044,0.132403
58331,58331.0,18.0,0.174859,0.149629,0.324487,4.523966,82.287666,77.763702,-3.144636,80.104828,-3.204978,3.133174,-3.111835,-4.501707,0.135931
61105,61105.0,19.0,0.103350,0.099540,0.202890,4.386719,83.305450,78.918732,-3.110238,90.265030,-3.194047,4.486143,-3.946888,-4.362458,0.135173
61734,61734.0,19.0,0.085382,0.122739,0.208121,5.616360,82.097198,76.480835,-3.001094,86.639343,-3.183582,4.776386,-3.373037,-5.607590,0.135700
61736,61736.0,19.0,0.165824,0.097346,0.263170,4.036482,82.085983,78.049500,-3.076244,85.171997,-3.183231,2.978262,-5.042030,-4.001179,0.136601
58873,58873.0,18.0,0.108463,0.078532,0.186996,3.581184,80.531845,76.950661,-3.185919,82.905205,-3.171715,5.596635,-5.635791,-3.510769,0.134595
61098,61098.0,19.0,0.169183,0.205668,0.374851,5.985293,85.531013,79.545723,-3.043691,89.682724,-3.162705,10.827540,-3.088485,-5.979549,0.132195
50194,50194.0,16.0,0.129758,0.179003,0.308761,4.607195,83.830231,79.223038,-3.058776,85.793510,-3.162474,4.483585,-3.046324,-4.589757,0.135605


In [16]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
58343,58343.0,18.0,0.119685,0.109013,0.228699,5.949080,85.189095,79.240013,-2.902843,81.792953,-3.210074,6.080492,-4.639206,-5.941076,0.135599
58877,58877.0,18.0,0.136434,0.233085,0.369519,5.046766,81.702965,76.656197,-3.139060,85.280731,-3.208382,5.029647,-2.584076,-5.034982,0.131992
61102,61102.0,19.0,0.115517,0.083937,0.199455,5.460420,84.645813,79.185394,-3.132429,84.146347,-3.206190,5.837498,-5.359791,-5.452044,0.132403
58331,58331.0,18.0,0.174859,0.149629,0.324487,4.523966,82.287666,77.763702,-3.144636,80.104828,-3.204978,3.133174,-3.111835,-4.501707,0.135931
61105,61105.0,19.0,0.103350,0.099540,0.202890,4.386719,83.305450,78.918732,-3.110238,90.265030,-3.194047,4.486143,-3.946888,-4.362458,0.135173
61734,61734.0,19.0,0.085382,0.122739,0.208121,5.616360,82.097198,76.480835,-3.001094,86.639343,-3.183582,4.776386,-3.373037,-5.607590,0.135700
61736,61736.0,19.0,0.165824,0.097346,0.263170,4.036482,82.085983,78.049500,-3.076244,85.171997,-3.183231,2.978262,-5.042030,-4.001179,0.136601
58873,58873.0,18.0,0.108463,0.078532,0.186996,3.581184,80.531845,76.950661,-3.185919,82.905205,-3.171715,5.596635,-5.635791,-3.510769,0.134595


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
18509,18509.0,5.0,0.216410,0.072366,0.288777,4.317364,84.764130,80.446762,-2.263197,74.716484,-2.565526,2.467461,-4.837002,-4.283128,0.144205
60970,60970.0,19.0,0.071270,0.114642,0.185912,4.566403,85.037079,80.470673,-1.972225,74.740784,-2.101667,4.101063,-4.239943,-4.545723,0.134633
61031,61031.0,19.0,0.098063,0.131593,0.229656,4.850329,85.275841,80.425514,-2.404906,74.838669,-2.451001,4.365208,-5.238818,-4.828303,0.133132
30403,30403.0,9.0,0.066475,0.172846,0.239321,5.164228,85.735298,80.571068,-2.494665,75.158997,-2.716716,4.425041,-2.841791,-5.153718,0.128267
45402,45402.0,14.0,0.087088,0.109626,0.196714,4.125506,81.111557,76.986053,-2.255456,75.319420,-2.097275,3.964150,-3.395751,-4.088764,0.130550
30717,30717.0,9.0,0.137256,0.177874,0.315129,3.879995,83.335442,79.455444,-2.521044,75.446121,-2.573784,3.459915,-3.126797,-3.839028,0.135283
30402,30402.0,9.0,0.168800,0.205257,0.374057,3.385212,83.207962,79.822754,-2.517479,75.454910,-2.388796,3.431785,-2.873969,-3.310751,0.131596
53811,53811.0,17.0,0.126854,0.109364,0.236218,4.191284,83.466110,79.274826,-2.382415,75.531853,-2.502528,5.192829,-3.550926,-4.157985,0.130698


In [17]:
df_metrics[df_metrics.step.isin([44861,44862,44863])]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
44861,44861.0,14.0,0.136696,0.113277,0.249972,4.054009,87.404732,83.350723,-1.874791,90.303818,-1.719401,3.460629,-4.420738,-4.004211,0.130161
44862,44862.0,14.0,0.129774,0.112522,0.242296,4.816566,85.825188,81.008621,-1.737367,90.969093,-1.970317,4.635601,-3.586227,-4.796669,0.130385
44863,44863.0,14.0,0.140365,0.111868,0.252234,4.557131,87.850899,83.293770,-1.916813,87.102127,-1.768400,3.836370,-4.101990,-4.534204,0.129952


## Read stored chi-squares for images

In [18]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting


col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [19]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [20]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [21]:
df_sliced=w.result
# df_sliced

In [22]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
# print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.048186995236436596, 'chi_spec1': 0.5455751014064505}
[0, 20]
{'chi_1': 0.17486642141356074, 'chi_spec1': 1.7635900897770111}
[0, 20]
{'chi_1': 0.048186995236436596, 'chi_spec1': 0.5455751014064505}
[22370 32820 39600 40610 52910 56280]


In [30]:
# best_step=[58800]
best_step=np.arange(32800,32900,10).astype(int)

In [31]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 22)
[(10, 32800), (10, 32810), (10, 32820), (10, 32830), (10, 32840), (10, 32850), (10, 32860), (10, 32870), (10, 32880), (10, 32890)]


In [32]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
3279,10-32800,0.031383,0.421403,33.203983,0.007847,0.020097,0.003439,0.001432,17.005208,9.277372,10,32800
3280,10-32810,0.010838,0.187553,31.471455,0.003672,0.005468,0.001699,0.002241,15.984835,7.036587,10,32810
3281,10-32820,0.007747,0.042310,30.564350,0.002823,0.002631,0.002292,0.000637,14.539451,2.096552,10,32820
3282,10-32830,0.012355,0.185210,31.035256,0.002598,0.007452,0.002306,0.001535,15.035417,9.706904,10,32830
3283,10-32840,0.043492,1.176091,30.627440,0.019933,0.019471,0.004088,0.005354,12.329044,48.967519,10,32840
3284,10-32850,0.031130,0.711066,29.303793,0.009840,0.015728,0.005562,0.002952,12.982280,28.087015,10,32850
3285,10-32860,0.194257,0.491715,35.831632,0.021480,0.094752,0.078026,0.006381,20.631841,5.225016,10,32860
3286,10-32870,0.025186,0.651716,30.440284,0.020472,0.002156,0.002557,0.016509,15.728439,37.467306,10,32870
3287,10-32880,0.037046,0.662741,28.796394,0.030178,0.003546,0.003322,0.008319,12.780940,52.780801,10,32880
3288,10-32890,0.028398,0.340878,31.670594,0.020260,0.004051,0.004087,0.008430,16.068692,20.056148,10,32890


### Plots 2: stored data

In [27]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [28]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [33]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('10-32800', '10-32810', '10-32820', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [29]:
# step=best_step[-1]
step=117600
fname='*gen_img_epoch-*_step-{0}.npy'.format(step)
# fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(result_dir+'/images/'+fname)[0]
print(fle)
imgs=np.load(fle)
print(imgs.shape)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201121_050116_full_loss_b64_lr0.0002_using-var/images/gen_img_epoch-37_step-117600.npy
(64, 128, 128)


In [30]:
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
df_metrics[df_metrics.step<2000]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
0,0.0,0.0,155.384521,8994.041016,9149.425781,0.000000,86.405716,86.405716,2.091570,NaN,NaN,10250.341797,9178.380859,8575.310547,16.994106
1,1.0,0.0,196.520554,9452.510742,9649.031250,0.000000,inf,inf,1.997464,inf,2.475530,10238.923828,9532.652344,9144.101562,0.465852
2,2.0,0.0,145.714554,9150.929688,9296.644531,0.000000,112.715851,112.715851,2.111522,inf,2.132113,10227.984375,9256.610352,8818.741211,0.399666
3,3.0,0.0,156.997803,8639.974609,8796.972656,0.000000,92.414574,92.414574,2.142157,104.134933,2.044554,10240.458984,8736.583008,8383.701172,0.405215
4,4.0,0.0,150.957550,8811.884766,8962.842773,0.000000,87.315872,87.315872,2.100545,87.325592,2.294785,10213.558594,8902.645508,8509.358398,0.397767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995.0,0.0,2.332118,22.379145,24.711262,88.450546,172.453705,84.003159,1.426435,84.166122,1.458471,73.462387,-8.319846,-88.012779,0.403768
1996,1996.0,0.0,7.975113,3.122581,11.097694,97.729805,181.735413,84.005600,1.542325,86.985519,1.418108,32.516716,-54.417641,-97.729805,0.415173
1997,1997.0,0.0,17.508060,6.264686,23.772747,48.942490,135.066467,86.123985,1.571042,85.655090,1.506533,-1.515057,-52.006775,-44.579414,0.417067
1998,1998.0,0.0,9.641037,7.489760,17.130798,68.521042,154.226471,85.705429,1.491595,92.041786,1.432904,26.949451,-83.883057,-63.937595,0.449832
